In [1]:
import os
import logging
from datetime import datetime
from mutagen.flac import FLAC
import time
import syncedlyrics
import random

In [ ]:
# Configuration
MUSIC_DIR = "C:\\Users\\Max\\Desktop\\music"  # ← Change this to your actual folder
LOG_FILE = "lyrics-fetch-log.txt"

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),
        logging.StreamHandler()  # also print to console
    ]
)

def main():
    logging.info("===== Starting synced lyrics fetcher =====")
    logging.info(f"Music directory: {MUSIC_DIR}")

    if not os.path.isdir(MUSIC_DIR):
        logging.error(f"Directory not found: {MUSIC_DIR}")
        return

    flac_files = [f for f in os.listdir(MUSIC_DIR) if f.lower().endswith(".flac")] #.mp3, .wav, .aac, .m4a
    total_files = len(flac_files)
    logging.info(f"Found {total_files} FLAC files")

    processed = 0
    success = 0
    skipped = 0
    failed = 0

    for i, filename in enumerate(flac_files, 1):
        file_path = os.path.join(MUSIC_DIR, filename)
        try:
            audio = FLAC(file_path)
            artist = audio.get("artist", [""])[0].strip()
            title = audio.get("title", [""])[0].strip()
            album = audio.get("album", [""])[0].strip()

            if not (artist and title):
                logging.warning(f"[{i}/{total_files}] Skipping {filename} - missing artist or title tag")
                skipped += 1
                continue

            processed += 1
            search_query = f"{title} {artist}"
            # if album:
            #     search_query += f" {album}"

            logging.info(f"[{i}/{total_files}] Searching: {filename}")
            lrc_path = os.path.join(MUSIC_DIR, os.path.splitext(filename)[0] + ".lrc")

            try:
                # Fetch lyrics using syncedlyrics API
                # Will return synced if available, else static/unsynced automatically
                lyrics_text = syncedlyrics.search(search_query)

                if lyrics_text and len(lyrics_text.strip()) > 10:
                    with open(lrc_path, "w", encoding="utf-8") as f:
                        f.write(lyrics_text)
                    logging.info(f"SUCCESS: {filename}")
                    success += 1
                else:
                    logging.info(f"FAILURE: {filename}")
                    failed += 1

            except Exception as e_lyrics:
                logging.error(f"Error fetching lyrics for {filename}: {str(e_lyrics)}")
                failed += 1
        
            # THIS DELAY BEFORE NEW REQUEST IS VERY IMPORTANT TO AVOID GETTING FLAGGED
            delay = random.uniform(11, 15)
            logging.info(f"sleeping for - {delay:.2f} seconds")
            time.sleep(delay)

        except Exception as e_file:
            logging.error(f"Error processing {filename}: {str(e_file)}")
            failed += 1

    # Final summary
    if processed > 0:
        success_rate = (success / processed) * 100
    else:
        success_rate = 0.0

    logging.info("===== Finished processing all files =====")
    logging.info(f"Summary:")
    logging.info(f"  Total FLAC files: {total_files}")
    logging.info(f"  Processed: {processed} (skipped {skipped} due to missing tags)")
    logging.info(f"  Lyrics found: {success}/{processed} ({success_rate:.1f}% success rate)")
    logging.info(f"Log saved to: {os.path.abspath(LOG_FILE)}")    



In [3]:
main()

2026-01-28 11:47:00,603 - INFO - ===== Starting synced lyrics fetcher =====
2026-01-28 11:47:00,605 - INFO - Music directory: C:\Users\Max\Desktop\music
2026-01-28 11:47:00,609 - INFO - Found 112 FLAC files
2026-01-28 11:47:00,618 - INFO - [1/112] Searching: 01 Ed Sheeran - Sapphire.flac
2026-01-28 11:47:04,312 - INFO - Lyrics found for "Sapphire Ed Sheeran" on Musixmatch
2026-01-28 11:47:04,328 - INFO - SUCCESS: 01 Ed Sheeran - Sapphire.flac
2026-01-28 11:47:04,328 - INFO - sleeping for - 12.99 seconds
2026-01-28 11:47:17,319 - INFO - [2/112] Searching: A.R. Rahman - Suhana Sama.flac
2026-01-28 11:47:20,732 - ERROR - An error occurred while searching for an LRC on Lrclib
2026-01-28 11:47:20,732 - ERROR - HTTPSConnectionPool(host='lrclib.net', port=443): Max retries exceeded with url: /api/search?q=Suhana+Sama+A.R.+Rahman (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
2026-01-28 11:47:30,954 - INFO - No su

KeyboardInterrupt: 